In [1]:
import os

# disable GPU

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import pandas as pd

from datasets import Dataset as HFDataset, Audio

In [3]:
def prep_df(root_path):
    # Read the TSV file and extract data
    tsv_file_path = os.path.join(root_path, "utt_spk_text.tsv")
    with open(tsv_file_path, "r", encoding="utf-8") as tsv_file:
        lines = tsv_file.readlines()

    # Prepare data for the Hugging Face dataset
    file_paths = []
    folder_names = []
    texts = []

    for line in lines:
        file_name, _, text = line.strip().split("\t")
        folder_name = file_name[:2]

        file_path = os.path.join(root_path, "data", folder_name, file_name + ".flac")

        file_paths.append(file_path)
        folder_names.append(folder_name)
        texts.append(text)

    # Create the Hugging Face dataset
    dataset_dict = {
        "file_path": file_paths,
        "folder_name": folder_names,
        "text": texts,
    }
    
    # create dataframe
    df = pd.DataFrame(dataset_dict)
    
    return df, dataset_dict


In [4]:
df,dataset_dict = prep_df("/home/venom/repo/Bengali.AI-Speech-Recognition/openslr/")

In [5]:
print(df.head())

                                           file_path folder_name  \
0  /home/venom/repo/Bengali.AI-Speech-Recognition...          00   
1  /home/venom/repo/Bengali.AI-Speech-Recognition...          00   
2  /home/venom/repo/Bengali.AI-Speech-Recognition...          00   
3  /home/venom/repo/Bengali.AI-Speech-Recognition...          00   
4  /home/venom/repo/Bengali.AI-Speech-Recognition...          00   

                      text  
0  বাংলাদেশে দায়িত্ব নেবে  
1      এ ধরণের কার্ড নিয়ে  
2        হতে উপার্জিত অর্থ  
3    হাসির বিষয় হয়েই আছে  
4          সার্ক দেশগুলোতে  


In [6]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor


# Load the tokenizer and feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="bengali", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="bengali", task="transcribe")

In [26]:
# Wrap the dictionary in a Hugging Face Dataset object
dataset_output = HFDataset.from_dict(dataset_dict)

# Cast the 'file_path' column to the 'Audio' feature
dataset = dataset_output.cast_column("file_path", Audio())

In [27]:
# test train split on dataset
# dataset = load_from_disk("./dataset")
dataset = dataset.train_test_split(test_size=0.00001)

In [7]:
import librosa
from sklearn.model_selection import train_test_split


fraction = 0.00001

num_samples = len(df)
num_test_samples = int(fraction * num_samples)
num_train_samples = num_samples - num_test_samples

# Split the dataset into train, test, and validation sets.
train_set, test_set = train_test_split(df, train_size=num_train_samples, random_state=42)

sampling_rate = 16000
#path_template = "/home/venom/repo/Bengali.AI-Speech-Recognition/{}"
path_template = "{}"

In [8]:
from datasets import IterableDataset

def dataset_generator(df):
    for _, row in df.iterrows():
        audio_array = librosa.load(path_template.format(row["file_path"]))[0]
        yield {
            "input_features": feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0], 
            "labels": tokenizer(row["text"]).input_ids
        }

In [9]:
train_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": train_set})
eval_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": test_set})

In [10]:
df

,file_path,folder_name,text
0,/home/venom/repo/Bengali.AI-Speech-Recognition...,00,বাংলাদেশে দায়িত্ব নেবে
1,/home/venom/repo/Bengali.AI-Speech-Recognition...,00,এ ধরণের কার্ড নিয়ে
2,/home/venom/repo/Bengali.AI-Speech-Recognition...,00,হতে উপার্জিত অর্থ
3,/home/venom/repo/Bengali.AI-Speech-Recognition...,00,হাসির বিষয় হয়েই আছে
4,/home/venom/repo/Bengali.AI-Speech-Recognition...,00,সার্ক দেশগুলোতে
...,...,...,...
218698,/home/venom/repo/Bengali.AI-Speech-Recognition...,ff,সংস্কৃতি মন্ত্রী আবুল কালাম আযাদও
218699,/home/venom/repo/Bengali.AI-Speech-Recognition...,ff,অভিযোগ রয়েছে
218700,/home/venom/repo/Bengali.AI-Speech-Recognition...,ff,আন্তোনিও দা মাগদালেনার
218701,/home/venom/repo/Bengali.AI-Speech-Recognition...,ff,মি. খানের


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_path', 'folder_name', 'text'],
        num_rows: 218700
    })
    test: Dataset({
        features: ['file_path', 'folder_name', 'text'],
        num_rows: 3
    })
})

In [42]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)



In [15]:
import evaluate

metric = evaluate.load("wer")


2023-07-27 16:14:46.469493: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 16:14:46.511731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/venom/miniconda3/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/venom/miniconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [16]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("bangla-speech-processing/BanglaASR") #,load_in_8bit=True)


In [18]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [19]:
# from peft import prepare_model_for_int8_training

# model = prepare_model_for_int8_training(model)

In [20]:
# import bitsandbytes as bnb

# def find_all_linear_names(model, bits=8):
#     cls = bnb.nn.Linear4bit if bits == 4 else (bnb.nn.Linear8bitLt if bits == 8 else torch.nn.Linear)
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, cls):
#             names = name.split('.')
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])


#     if 'lm_head' in lora_module_names: # needed for 16-bit
#         lora_module_names.remove('lm_head')
#     return list(lora_module_names)

In [21]:
#target_modules = find_all_linear_names(model)
target_modules = ['k_proj', 'fc2', 'q_proj', 'fc1', 'out_proj', 'v_proj']
target_modules = ['q_proj', 'v_proj']


In [22]:
from peft import LoraConfig,LoraConfig, get_peft_model

config = LoraConfig(r=8, lora_alpha=32, target_modules=target_modules, lora_dropout=0.1, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 884,736 || all params: 242,619,648 || trainable%: 0.36465966680489126


In [43]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-bn",  # change to a repo name of your choice
    #per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=30,
    max_steps=1000,
    do_eval=True,
#   gradient_checkpointing=True,
    #fp16=True,
    use_ipex=True,
    bf16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    generation_max_length=225,
    save_steps=100,
    eval_steps=1,
    logging_steps=25,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    save_total_limit=5,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

In [44]:
from transformers import Seq2SeqTrainer


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    #train_dataset=train_ds,
    eval_dataset=dataset['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    
)
model.config.use_cache = False

In [45]:
# evaluate the model firstbt
trainer.evaluate()

[{'file_path': {'path': '/home/venom/repo/Bengali.AI-Speech-Recognition/openslr/data/27/278209a690.flac', 'array': array([-0.00866699, -0.00967407, -0.00994873, ...,  0.0090332 ,
        0.00793457,  0.00793457]), 'sampling_rate': 16000}, 'folder_name': '27', 'text': 'তাহলে চাকরিতে'}]


ValueError: You should supply an instance of `transformers.BatchFeature` or list of `transformers.BatchFeature` to this method that includes input_features, but you provided ['file_path']